In [1]:
import pandas as pd

In [2]:
job_data = pd.read_csv('linkdin_Job_data.csv')

In [3]:
job_data.head()

,job_ID,job,location,company_id,company_name,work_type,full_time_remote,no_of_employ,no_of_application,posted_day_ago,alumni,Hiring_person,linkedin_followers,hiring_person_link,job_details,Column1
0,3471657636,"Data Analyst, Trilogy (Remote) - $60,000/year USD","Delhi, Delhi, India",NaN,Crossover,Remote,Full-time · Associate,"1,001-5,000 employees · IT Services and IT Con...",200,8 hours,12 company alumni,NaN,"5,395,547 followers",NaN,About the job Crossover is the world's #1 sour...,NaN
1,3471669068,"Data Analyst, Trilogy (Remote) - $60,000/year USD","New Delhi, Delhi, India",NaN,Crossover,Remote,Full-time · Associate,"1,001-5,000 employees · IT Services and IT Con...",184,8 hours,12 company alumni,NaN,"5,395,547 followers",NaN,About the job Crossover is the world's #1 sour...,NaN
2,3474349934,Data Analyst - WFH,Greater Bengaluru Area,NaN,Uplers,Remote,Full-time · Mid-Senior level,"1,001-5,000 employees · IT Services and IT Con...",200,9 hours,3 company alumni,Shahid Ahmad,NaN,https://www.linkedin.com/in/shahid-ahmad-a2613...,About the job Profile: ML EngineersExperience:...,NaN
3,3472816027,Data Analyst,"Gurugram, Haryana, India",NaN,PVAR SERVICES,On-site,Full-time,1-10 employees,200,7 hours,NaN,Vartika Singh,"2,094 followers",https://www.linkedin.com/in/vartika-singh-,About the job Designation: Data AnalystLocatio...,NaN
4,3473311511,Data Analyst,"Mohali district, Punjab, India",NaN,Timeline Freight Brokers,On-site,Full-time,1-10 employees,8,26 minutes,1 company alumni,Manisha (Gisele Smith),NaN,https://www.linkedin.com/in/manisharathore0029,About the job The ideal candidate will use the...,NaN


In [4]:
job_ratings = pd.read_csv('job raings.csv')

In [5]:
job_ratings.head()

,user_id,job_post,company,user ratings,company_review,exp_required,salary_offered,job_location,job_description,required_skills,Posted_as_on_22_5_2022
0,1,Information Technology Executive,Marriott,4.2,(2907 Reviews),0-3 Yrs,Not disclosed,Bhopal,POSITION SUMMARY Troubleshoot and resolve tech...,Supervisor\nAdministration\nTactical planning\...,5 DAYS AGO
1,2,Associate Director-Information Technology(SAP),Unacademy,3.4,(544 Reviews),5-15 Yrs,Not disclosed,"Noida, Greater Noida",Support / manage timely delivery of custom dev...,Relationship management\nSAP MM\nPharma\nAnaly...,3 DAYS AGO
2,3,Lead - Information Technology,EchoIndia,4.0,(20 Reviews),11-13 Yrs,Not disclosed,Delhi / NCR,Experience in designing modern REST APIs that ...,Spring\nDocker\nSystem Design\nPaaS\nMicroserv...,1 DAY AGO
3,4,Information Technology - Project Manager,Tata International,4.1,(287 Reviews),4-8 Yrs,Not disclosed,Mumbai (All Areas),Should be able to manage a project using vario...,project management\nit projects\ninformation t...,2 DAYS AGO
4,10,Information Technology Trainee,Stefanini,3.8,(75 Reviews),2-7 Yrs,Not disclosed,Noida,The resource will be expected to cross skill\t...,POP\nTraining\nBasic\nNetworking\nIntern\nIT i...,4 DAYS AGO


In [6]:
job_data["job_details"].head()

0    About the job Crossover is the world's #1 sour...
1    About the job Crossover is the world's #1 sour...
2    About the job Profile: ML EngineersExperience:...
3    About the job Designation: Data AnalystLocatio...
4    About the job The ideal candidate will use the...
Name: job_details, dtype: object

In [7]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import string

In [8]:
def preprocess_text(text):
   
    text = text.lower()
    
    text = text.translate(str.maketrans('', '', string.punctuation))
   
    tokens = text.split()
    
    tokens = [word for word in tokens if word not in ENGLISH_STOP_WORDS]
    return " ".join(tokens)

In [9]:
job_data['job_details_clean'] = job_data['job_details'].fillna('').apply(preprocess_text)

In [10]:
job_data['job_details_clean'].head()

0    job crossover worlds 1 source fulltime remote ...
1    job crossover worlds 1 source fulltime remote ...
2    job profile ml engineersexperience 3 yearsloca...
3    job designation data analystlocation gurgaonsa...
4    job ideal candidate use passion big data analy...
Name: job_details_clean, dtype: object

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
tfidf_vectorizer = TfidfVectorizer()

In [13]:
tfidf_matrix = tfidf_vectorizer.fit_transform(job_data['job_details_clean'])

In [14]:
tfidf_matrix.shape

(7927, 59898)

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
cosine_similarity_matrix = cosine_similarity(tfidf_matrix)

In [17]:
cosine_similarity_matrix.shape

(7927, 7927)

In [18]:
def recommend_jobs_based_on_content(job_ID, top_n=5):
    
    job_idx = job_data[job_data['job_ID'] == job_ID].index[0]
    
    
    similarity_scores = cosine_similarity_matrix[job_idx]
    
    
    similar_jobs_idx = similarity_scores.argsort()[-top_n-1:-1][::-1]
    
    
    similar_jobs = job_data.iloc[similar_jobs_idx]
    
    return similar_jobs[['job_ID', 'job', 'location', 'company_name']]

In [19]:
sample_job_ID = job_data['job_ID'][0] 
recommendations_content_based = recommend_jobs_based_on_content(sample_job_ID)
recommendations_content_based

,job_ID,job,location,company_name
283,3471657636,"Data Analyst, Trilogy (Remote) - $60,000/year USD","Delhi, Delhi, India",Crossover
1773,3471657636,"Data Analyst, Trilogy (Remote) - $60,000/year USD","Delhi, Delhi, India",Crossover
3799,3471657636,"Data Analyst, Trilogy (Remote) - $60,000/year USD","Delhi, Delhi, India",Crossover
5848,3471657636,"Data Analyst, Trilogy (Remote) - $60,000/year USD","Delhi, Delhi, India",Crossover
771,3471657636,"Data Analyst, Trilogy (Remote) - $60,000/year USD","Delhi, Delhi, India",Crossover


In [20]:
job_data_no_duplicates = job_data.drop_duplicates(subset=['job_ID'])

In [21]:
job_data_no_duplicates.shape

(5843, 17)

In [22]:
tfidf_matrix_no_duplicates = tfidf_vectorizer.transform(job_data_no_duplicates['job_details_clean'])

In [23]:
cosine_similarity_matrix_no_duplicates = cosine_similarity(tfidf_matrix_no_duplicates)

In [24]:
def recommend_jobs_based_on_content_no_duplicates(job_ID, top_n=5):
    
    job_idx = job_data_no_duplicates[job_data_no_duplicates['job_ID'] == job_ID].index[0]
    
    
    similarity_scores = cosine_similarity_matrix_no_duplicates[job_idx]
    
    similar_jobs_idx = similarity_scores.argsort()[-top_n-1:-1][::-1]
    
    similar_jobs = job_data_no_duplicates.iloc[similar_jobs_idx]
    
    return similar_jobs[['job_ID', 'job', 'location', 'company_name']]

In [25]:
recommendations_content_based_no_duplicates = recommend_jobs_based_on_content_no_duplicates(sample_job_ID)
recommendations_content_based_no_duplicates

,job_ID,job,location,company_name
1,3471669068,"Data Analyst, Trilogy (Remote) - $60,000/year USD","New Delhi, Delhi, India",Crossover
6,3471658510,"Data Analyst, Trilogy (Remote) - $60,000/year USD","Bengaluru, Karnataka, India",Crossover
5885,3471662372,"Analyst, Trilogy (Remote) - $60,000/year USD","Delhi, Delhi, India",Crossover
219,3471676193,"Operations Analyst, Trilogy (Remote) - $30,000...","Kolkata, West Bengal, India",Crossover
1376,3471677468,"Operations Analyst, Trilogy (Remote) - $30,000...","Chennai, Tamil Nadu, India",Crossover


In [26]:
top_jobs = job_ratings['job_post'].value_counts().head(1000).index


top_users = job_ratings['user_id'].value_counts().head(1000).index

In [27]:
filtered_job_ratings = job_ratings[job_ratings['job_post'].isin(top_jobs) & job_ratings['user_id'].isin(top_users)]

In [28]:
filtered_user_job_matrix = filtered_job_ratings.pivot_table(index='user_id', columns='job_post', values='user ratings ', fill_value=0)

In [29]:
filtered_user_job_matrix_shape = filtered_user_job_matrix.shape
filtered_user_job_matrix_head = filtered_user_job_matrix.head()

filtered_user_job_matrix_shape, filtered_user_job_matrix_head

((381, 155),
 job_post  Android Engineer  Angular Developer  \
 user_id                                         
 42576                    4                0.0   
 42684                    0                0.0   
 42687                    0                0.0   
 42691                    0                0.0   
 42697                    0                0.0   
 
 job_post  Application Database Administrator  \
 user_id                                        
 42576                                      0   
 42684                                      0   
 42687                                      0   
 42691                                      0   
 42697                                      0   
 
 job_post  Application Developer: AWS Cloud Microservices  \
 user_id                                                    
 42576                                                0.0   
 42684                                                0.0   
 42687                                        

In [30]:
user_similarity_matrix = cosine_similarity(filtered_user_job_matrix)


user_similarity_df = pd.DataFrame(user_similarity_matrix, index=filtered_user_job_matrix.index, columns=filtered_user_job_matrix.index)

In [31]:
user_similarity_shape = user_similarity_df.shape
user_similarity_head = user_similarity_df.head()

user_similarity_shape, user_similarity_head

((381, 381),
 user_id  42576  42684  42687  42691  42697  42699  42701  42707  42714  42715  \
 user_id                                                                         
 42576      1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 42684      0.0    1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 42687      0.0    0.0    1.0    1.0    0.0    0.0    0.0    0.0    0.0    0.0   
 42691      0.0    0.0    1.0    1.0    0.0    0.0    0.0    0.0    0.0    0.0   
 42697      0.0    0.0    0.0    0.0    1.0    0.0    0.0    0.0    0.0    0.0   
 
 user_id  ...  46081  46084  46086  46095  46104  46125  46126  46129  46139  \
 user_id  ...                                                                  
 42576    ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 42684    ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 42687    ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 42691    .

In [34]:
def recommend_jobs_based_on_collaborative(user_id, top_n=5):
    
    user_similarity_scores = user_similarity_df.loc[user_id]
    
    
    predicted_interest_dict = {}
    
    
    for job in filtered_user_job_matrix.columns:
        
        if filtered_user_job_matrix.loc[user_id, job] != 0:
            continue
        
        predicted_interest_dict[job] = sum(user_similarity_scores * filtered_user_job_matrix[job]) / sum(user_similarity_scores)
    
    
    recommended_jobs = sorted(predicted_interest_dict, key=predicted_interest_dict.get, reverse=True)[:top_n]
    
    return recommended_jobs

In [35]:
recommendations_collaborative_based_fixed = recommend_jobs_based_on_collaborative(sample_user_ID)
recommendations_collaborative_based_fixed

['Angular Developer',
 'Application Database Administrator',
 'Application Developer: AWS Cloud Microservices',
 'Application Developer: Google Cloud Microservices',
 'Application Developer: RPA']

In [36]:
def hybrid_recommendation_system(user_id, job_ID, top_n=5):
    
    content_based_recommendations = recommend_jobs_based_on_content_no_duplicates(job_ID, top_n)
    
    
    collaborative_based_recommendations = recommend_jobs_based_on_collaborative(user_id, top_n)
    
    
    hybrid_recommendations = content_based_recommendations['job'].tolist() + collaborative_based_recommendations
    
    
    hybrid_recommendations = list(set(hybrid_recommendations))
    
    
    hybrid_recommendations = hybrid_recommendations[:top_n]
    
    return hybrid_recommendations

In [37]:
sample_user_ID_hybrid = sample_user_ID  
sample_job_ID_hybrid = sample_job_ID    

hybrid_recommendations = hybrid_recommendation_system(sample_user_ID_hybrid, sample_job_ID_hybrid)
hybrid_recommendations

['Analyst, Trilogy (Remote) - $60,000/year USD',
 'Application Developer: AWS Cloud Microservices',
 'Application Database Administrator',
 'Application Developer: RPA',
 'Data Analyst, Trilogy (Remote) - $60,000/year USD']